In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df=pd.read_csv("heart_clean_final.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,62,0,0,138,294.0,1,1,106,0,1.9,1,3,2,0
1,58,0,0,100,248.0,0,0,122,0,1.0,1,0,2,1
2,58,1,0,114,318.0,0,2,140,0,3.5,0,3,1,0
3,54,1,0,122,286.0,0,0,116,1,3.2,1,2,2,0
4,71,0,0,112,149.0,0,1,125,0,1.6,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,60,0,2,120,178.0,1,1,100,0,0.0,2,0,2,1
179,64,1,2,140,335.0,0,1,158,0,0.0,2,0,2,0
180,68,0,2,120,211.0,0,0,115,0,1.5,1,0,2,1
181,44,0,2,108,141.0,0,1,175,0,0.6,1,0,2,1


In [5]:
indep=df.drop("target",axis=1)
dep=df["target"]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [61]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy','log_loss'],'max_features': ['sqrt','log2'],'splitter':['best','random'],'max_depth':[5,10,20,30],'min_samples_split':[2,5,10],'min_samples_leaf':[1,2,4,10]} 
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4, 10],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [62]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [63]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'log_loss', 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}: 0.8507776376628836


In [78]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[ 5  7]
 [ 1 48]]


In [80]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.83      0.42      0.56        12
           1       0.87      0.98      0.92        49

    accuracy                           0.87        61
   macro avg       0.85      0.70      0.74        61
weighted avg       0.86      0.87      0.85        61



In [82]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.7840136054421769

In [84]:
table=pd.DataFrame.from_dict(re)

In [86]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001109,0.000492,0.001826,4.082881e-04,gini,5,sqrt,1,2,best,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.760000,0.633143,0.799968,0.639943,0.631818,0.692974,0.072211,494
1,0.001012,0.000011,0.002205,4.102709e-04,gini,5,sqrt,1,2,random,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.785031,0.871019,0.863022,0.642015,0.871978,0.806613,0.088500,16
2,0.001400,0.000490,0.002166,3.298297e-04,gini,5,sqrt,1,5,best,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.796190,0.806112,0.833333,0.648148,0.479918,0.712740,0.133119,415
3,0.001201,0.000400,0.001999,9.702201e-07,gini,5,sqrt,1,5,random,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.785031,0.785031,0.736111,0.559930,0.562462,0.685713,0.103228,513
4,0.000900,0.000491,0.001903,4.928882e-04,gini,5,sqrt,1,10,best,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.796190,0.843889,0.648148,0.648148,0.833333,0.753942,0.087821,176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,0.000997,0.000005,0.001603,5.862258e-04,log_loss,30,log2,10,2,random,"{'criterion': 'log_loss', 'max_depth': 30, 'ma...",0.785031,0.760000,0.808772,0.747151,0.544872,0.729165,0.094548,320
572,0.000800,0.000400,0.001603,5.860954e-04,log_loss,30,log2,10,5,best,"{'criterion': 'log_loss', 'max_depth': 30, 'ma...",0.871019,0.920000,0.747151,0.680385,0.631818,0.770075,0.109865,98
573,0.000400,0.000490,0.001703,6.038822e-04,log_loss,30,log2,10,5,random,"{'criterion': 'log_loss', 'max_depth': 30, 'ma...",0.833016,0.749524,0.808772,0.583333,0.736111,0.742151,0.087188,249
574,0.000800,0.000400,0.001001,6.324142e-04,log_loss,30,log2,10,10,best,"{'criterion': 'log_loss', 'max_depth': 30, 'ma...",0.582540,0.600000,0.786630,0.677083,0.824074,0.694065,0.097000,490
